In [1]:
from keras import backend as K
import tensorflow as tf
from math import ceil
import numpy as np
import math
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy32
from bounding_box_utils.bounding_box_utils import iou_float, convert_coordinates
import timeit
import numpy as np

Using TensorFlow backend.


In [2]:
def np_intersection_area_(boxes1, boxes2, coords='corners', mode='outer_product', border_pixels='half'):
    '''
    The same as 'intersection_area()' but for internal use, i.e. without all the safety checks.
    '''

    m = boxes1.shape[0] # The number of boxes in `boxes1`
    n = boxes2.shape[0] # The number of boxes in `boxes2`

    # Set the correct coordinate indices for the respective formats.
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3
    
    d = 0
    # Compute the intersection areas.    
    
    min_xy = np.maximum(np.tile(np.expand_dims(boxes1[:,[xmin,ymin]], axis=1), reps=(1, n, 1)),
                        np.tile(np.expand_dims(boxes2[:,[xmin,ymin]], axis=0), reps=(m, 1, 1)))

    # For all possible box combinations, get the smaller xmax and ymax values.
    # This is a tensor of shape (m,n,2).
    max_xy = np.minimum(np.tile(np.expand_dims(boxes1[:,[xmax,ymax]], axis=1), reps=(1, n, 1)),
                        np.tile(np.expand_dims(boxes2[:,[xmax,ymax]], axis=0), reps=(m, 1, 1)))
    
    # Compute the side lengths of the intersection rectangles.
    side_lengths = np.maximum(0, max_xy - min_xy + d)
    return side_lengths[:,:,0] * side_lengths[:,:,1]

In [3]:
def iou_float(boxes1, boxes2, coords='centroids', mode='outer_product', border_pixels='half'):

    intersection_areas = np_intersection_area_(boxes1, boxes2, coords=coords, mode=mode)
    
    m = boxes1.shape[0] # The number of boxes in `boxes1`
    n = boxes2.shape[0] # The number of boxes in `boxes2`

    # Compute the union areas.

    # Set the correct coordinate indices for the respective formats.
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    if border_pixels == 'half':
        d = 0
    elif border_pixels == 'include':
        d = 1 # If border pixels are supposed to belong to the bounding boxes, we have to add one pixel to any difference `xmax - xmin` or `ymax - ymin`.
    elif border_pixels == 'exclude':
        d = -1 # If border pixels are not supposed to belong to the bounding boxes, we have to subtract one pixel from any difference `xmax - xmin` or `ymax - ymin`.

    if mode == 'outer_product':

        boxes1_areas = np.tile(np.expand_dims((boxes1[:,xmax] - boxes1[:,xmin] + d) * (boxes1[:,ymax] - boxes1[:,ymin] + d), axis=1), reps=(1,n))
        boxes2_areas = np.tile(np.expand_dims((boxes2[:,xmax] - boxes2[:,xmin] + d) * (boxes2[:,ymax] - boxes2[:,ymin] + d), axis=0), reps=(m,1))

    elif mode == 'element-wise':

        boxes1_areas = (boxes1[:,xmax] - boxes1[:,xmin] + d) * (boxes1[:,ymax] - boxes1[:,ymin] + d)
        boxes2_areas = (boxes2[:,xmax] - boxes2[:,xmin] + d) * (boxes2[:,ymax] - boxes2[:,ymin] + d)

    union_areas = boxes1_areas + boxes2_areas - intersection_areas
    op = intersection_areas / union_areas
    return op.astype(np.float32)


In [4]:
y = np.load('y_pred.npy')
x = np.load('y_true.npy')

f = iou_float(x,y,coords='centroids', mode='outer_product', border_pixels='half')

In [10]:
def match_bipartite_greedy32(weight_matrix):

    weight_matrix = np.copy(weight_matrix) # We'll modify this array.
    num_ground_truth_boxes = weight_matrix.shape[0]
    all_gt_indices = list(range(num_ground_truth_boxes)) # Only relevant for fancy-indexing below.
    # This 1D array will contain for each ground truth box the index of
    # the matched anchor box.
    matches = np.zeros(num_ground_truth_boxes, dtype=np.int)
    counter = 0
    print(num_ground_truth_boxes)
    for _ in range(num_ground_truth_boxes):
        anchor_indices = np.argmax(weight_matrix, axis=1) # Reduce along the anchor box axis.
        overlaps = weight_matrix[all_gt_indices, anchor_indices]

        ground_truth_index = np.argmax(overlaps) # Reduce along the ground truth box axis.

        anchor_index = anchor_indices[ground_truth_index]

        matches[ground_truth_index] = anchor_index # Set the match.
        # Set the row of the matched ground truth box and the column of the matched
        # anchor box to all zeros. This ensures that those boxes will not be matched again,
        # because they will never be the best matches for any other boxes.
        weight_matrix[ground_truth_index] = 0
#         print("weight_matrix: ",weight_matrix[:,anchor_index])
        weight_matrix[:,anchor_index] = 0

    return matches.astype(np.int32)


In [11]:
f

array([[-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -0., -0., -0.],
       [ 0., -0.,  0., ..., -0., -0.,  0.],
       [-0., -0., -0., ..., -0., -0., -0.]], dtype=float32)

In [12]:
match_bipartite_greedy32(f)

50


array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18,  0,  0,  0,  0,  0,
        0,  0, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 45,  0, 26, 35],
      dtype=int32)

In [7]:
x = match_bipartite_greedy32(f)

ground_truth_index:  30
anchor_index:  16


In [8]:
x[1]

0

In [44]:
t = np.ones((5,5))

In [45]:
t[:,1].assign(0)

AttributeError: 'numpy.ndarray' object has no attribute 'assign'

In [43]:
t

array([[1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.]])